# Bring data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#bring your data!
path = input('please type your path to the database: ')
data = pd.read_csv(f'{path}')
data.head()

#/Users/danielriojas/Documents/Random_2022/clean_depressionvspossitive.csv
#/Users/danielriojas/Documents/Random_2022/df_limpio.csv

please type your path to the database: /Users/danielriojas/Documents/Random_2022/df_limpio.csv


/Users/danielriojas/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ids,Tweet,label
0,1467822272,love u guys r best,0
1,1467822273,im meting one besties tonight cant wait girl talk,0
2,1467822283,thanks twiter ad sunisa got met hin show dc ar...,0
3,1467822287,sick realy cheap hurts much eat real fod plus ...,0
4,1467822293,efect everyone,0


In [4]:
positive_data = data[data.label == 0]

In [5]:
positive_data.shape

(800000, 3)

In [6]:
depression_data = data[data.label==1]

In [7]:
depression_data.shape

(2345, 3)

In [8]:
sample_positive_data = positive_data.sample( n  = 2345)

In [9]:
sample_data = sample_positive_data.append(depression_data)

In [10]:
sample_data =  sample_data.sample(frac=1)

In [11]:
sample_data.shape

(4690, 3)

In [12]:
text_col = input('choose the text column of your data: ')
X = sample_data[f'{text_col}']

target_col = input('choose the target column of your data: ')
y = sample_data[f'{target_col}']

choose the text column of your data: Tweet
choose the target column of your data: label


# naive bayes model

## cross validation

In [ ]:
!pip install nltk

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [14]:
tfid = TfidfVectorizer(ngram_range=(4,5))
nb = MultinomialNB()

In [15]:
X = X.apply(str)
#X.apply(word_tokenize)
X

801387    k debt vs k hapines vs depresion emoji face te...
559241                                        thanks wishes
800383    depresion anxiety constant state remembering e...
801434                           depresion let work healthy
800084    uncle tupelo depresion demos rsd record store ...
                                ...                        
800929    lack sexual interest symptom depresionme expla...
800333    last count received nearly mesages people teli...
745814                                      welcome namaste
801739    sigh mum wrong tiredmum tired even done anythi...
220491    aha pet siting bles ben nan hospital realy lis...
Name: Tweet, Length: 4690, dtype: object

In [16]:
vector = tfid.fit_transform(X)

In [ ]:
#X = pd.DataFrame(X.toarray(),columns = tfid.get_feature_names_out())

In [17]:
simple_cross_val_recall = cross_val_score(nb, vector , y , scoring= 'recall', cv=3,
               n_jobs = -1 , verbose = 1
               )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    3.1s finished


In [18]:
print('the recall: ' ,np.mean(simple_cross_val_recall))

the recall:  0.12199641310624344


In [19]:
simple_cross_val_accuracy = cross_val_score(nb, vector , y , scoring= 'accuracy', cv=3,
               n_jobs = -1 , verbose = 1
               )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.8s finished


In [20]:
print('the accuracy: ' , np.mean(simple_cross_val_accuracy) )

the accuracy:  0.5164125484959903


## grid search

In [21]:
tfid2 = TfidfVectorizer(ngram_range=(4,5))
nb2 = MultinomialNB()
X = X.apply(str)
X

801387    k debt vs k hapines vs depresion emoji face te...
559241                                        thanks wishes
800383    depresion anxiety constant state remembering e...
801434                           depresion let work healthy
800084    uncle tupelo depresion demos rsd record store ...
                                ...                        
800929    lack sexual interest symptom depresionme expla...
800333    last count received nearly mesages people teli...
745814                                      welcome namaste
801739    sigh mum wrong tiredmum tired even done anythi...
220491    aha pet siting bles ben nan hospital realy lis...
Name: Tweet, Length: 4690, dtype: object

In [22]:
pipe = Pipeline([
    ('TfidfVectorizer', tfid2),
    ('MultinomialNB()' , nb2)
    
])

In [23]:
pipe.get_params()

{'memory': None,
 'steps': [('TfidfVectorizer',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(4, 5), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=None, use_idf=True, vocabulary=None)),
  ('MultinomialNB()',
   MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
 'verbose': False,
 'TfidfVectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=None,
                 min_df=1, ngram_range=(4, 5), norm='l2', preprocessor=None,
              

In [24]:
pipe_grid = {
    'TfidfVectorizer__ngram_range': [(1,2) , (2,3), (3,4),(4, 5)],
    'MultinomialNB()__alpha': [0.1 , 0.5 , 1.0]
    
}

In [25]:
search_recall= GridSearchCV(pipe,
    pipe_grid,
    scoring='recall',
    n_jobs=-1,
    cv=3,
    verbose=1)

In [26]:
search_recall.fit(X , y )

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    5.3s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('TfidfVectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(4, 5),
                                               

In [27]:
print('the best params for recall: ',search_recall.best_params_)
print('the best recall score: ' , search_recall.best_score_)

the best params for recall:  {'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}
the best recall score:  0.9628981359286463


In [28]:
best_model_recall = search_recall.best_estimator_

In [32]:
best_model_recall["MultinomialNB()"].get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [30]:
search_accuracy= GridSearchCV(pipe,
    pipe_grid,
    scoring='accuracy',
    n_jobs=-1,
    cv=3,
    verbose=1)

In [31]:
search_accuracy.fit(X , y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    5.5s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('TfidfVectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(4, 5),
                                               

In [33]:
print('the best params for accuracy: ',search_accuracy.best_params_)
print('the best accuracy score: ' , search_accuracy.best_score_)

the best params for accuracy:  {'MultinomialNB()__alpha': 0.5, 'TfidfVectorizer__ngram_range': (1, 2)}
the best accuracy score:  0.9040507549093242


# testeo y analisis del modelo

In [ ]:
'''{'MultinomialNB()__alpha': 1.0, 'TfidfVectorizer__ngram_range': (1, 2)}'''

In [34]:
tfid3 = TfidfVectorizer(ngram_range=(1,2))
nb3 = MultinomialNB(alpha = 0.5)

In [35]:
tfid3.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [36]:
vector3 = tfid3.transform(X)

In [37]:
tfid3.get_feature_names()

['ab',
 'ab chalenges',
 'ab months',
 'ab riper',
 'abandoned',
 'abandoned heartbroken',
 'abatement',
 'abatement depresion',
 'abdula',
 'abdula shaikh',
 'abe',
 'abe lincoln',
 'abel',
 'abel love',
 'ability',
 'ability manage',
 'able',
 'able acomplish',
 'able aford',
 'able catch',
 'able come',
 'able dry',
 'able get',
 'able leave',
 'able pay',
 'able pul',
 'able runing',
 'able slepdepresion',
 'able take',
 'able translate',
 'able turn',
 'able wear',
 'able without',
 'able work',
 'able write',
 'abnormal',
 'abnormal functional',
 'aboard',
 'aboard email',
 'abort',
 'abort child',
 'abortion',
 'abortion child',
 'abortion considered',
 'abraham',
 'abraham lincoln',
 'abroad',
 'abroad depresion',
 'abs',
 'abs dayum',
 'absence',
 'absence cedar',
 'absolut',
 'absolut keine',
 'absolutely',
 'absolutely cream',
 'absolutely due',
 'absolutely impresed',
 'absolutely love',
 'absolutely nite',
 'absolutely reason',
 'absolutely shit',
 'absolutely stuning',
 '

In [38]:
nb3.fit(vector3, y)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [39]:
nb3.coef_

array([[-10.59542864, -10.74808071, -10.78606749, ..., -11.01818795,
        -10.76319344, -10.73295398]])

In [40]:
coefs = pd.Series(nb3.coef_[0], index = tfid3.get_feature_names())


In [41]:
coefs.sort_values()

#agregar palabras a las stopwords , por ejemplo 'twitter', 'com'
#agregar precision, accuracy, recall , f1 , etc
#min, max 
#max_features:overfitting? ~ regularizacion

#to do: traduccion?
#red de deeplearning para buscar mejor score
#testeos
#

hapy hapy       -11.018188
film trip       -11.018188
filming         -11.018188
filming media   -11.018188
films realy     -11.018188
                   ...    
face             -7.206142
emoji            -6.840789
nan              -6.700700
anxiety          -6.683061
depresion        -5.266215
Length: 41233, dtype: float64

In [ ]:
# 12345